In [ ]:
import folium
import os

import numpy as np

import pickle



In [ ]:
zipcode_file = '../../../CARES_tools/CARES_zipcode_data_v3.pkl'

with open(zipcode_file,'rb') as fin:
    zipcodes = pickle.load(fin,encoding='latin')[2]

def latlon(myzip):

    idx = zipcodes['ZipCode'] == myzip
  
    if len(idx[idx])==0:
        return None,None

    lat = zipcodes['Latitude'][idx][0]
    lon = zipcodes['Longitude'][idx][0]
  
    return lat,lon


In [ ]:
infilename = 'Admitted for Fall 2019 - Physics Majors - Sheet1.tsv'
# 2018
#infilename = 'Admits_Physics.xlsx - Export.tsv'

zips = []
names = []
schools = []
majors = []

infile = open(infilename,'r')

for i,line in enumerate(infile):
    
    if i==0:
        continue
    
    row = line.split('\t')
    z = row[10]
  
    if z == '':
        continue
    
    if z.find('-')>=0:
        zips.append(z.split('-')[0])
    elif len(z)==4:
        z = "0{0}".format(z)
        zips.append(z)
    else:
        zips.append(z)
    
    name = row[0] +" " + row[2]
    names.append(name)
    schools.append(row[13])
    majors.append(row[11])

#print(names)
#print(zips)
#print(schools)

print(len(names), len(zips), len(schools), len(majors))


In [ ]:
def popup(name, major, school):

    html="""
        <b>{0}</b><br><br>
        {1}<br><br>
        {2}
        <p>
    """.format(name,major,school)
    
    iframe = folium.IFrame(html=html, width=200, height=150)
    popup = folium.Popup(iframe, max_width=2650)
    
    return popup

In [ ]:

myzip = '12211'
lat,lon = latlon(myzip)

m = folium.Map(location=[lat, lon], zoom_start=7)
print(lat,lon)

# Siena
folium.CircleMarker(location=[lat, lon], radius=25, popup='Siena', fill_opacity=1.0, fill_color='yellow',color='yellow').add_to(m)

#'''
for i,(z,name,school,major) in enumerate(zip(zips,names,schools,majors)):
  
    #print("Hi there: ",z)
    #z = zipcode.isequal(z)
    #print(z)
  
    # For some reason, it doesn't like ' in the popups
    name = name.replace('\'','')

    '''
    color = 'red'
    if school != '':
        color = 'orange'
    else:
        color = 'blue'
    ''' 
    color = 'red'
    if major.find('Education')>=0:
        color = 'red'
    elif major.find('Applied')>=0:
        color = 'pink'
    elif major.find('Engineering')>=0:
        color = 'beige'
    else:
        color = 'blue'

        
    if z is not None:
        label = popup(name,major,school)
        #print(i,z,name,school)
        lat,lon = latlon(z)
        
        if lat is None:
            continue 
            
        lat += 0.025*np.random.random()
        lon += 0.025*np.random.random()
        '''
        if major.find('Applied')>=0:
            folium.CircleMarker(location=[lat,lon], popup=label , fill_opacity=0.0,radius=10).add_to(m)
        elif major.find('Education')>=0:
            folium.CircleMarker(location=[lat,lon], popup=label , fill_color=color,radius=10).add_to(m)
        else:
            folium.Marker(location=[lat,lon], popup=label ,icon=folium.Icon(color=color, icon='ok-sign'),).add_to(m)
        '''
        folium.Marker(location=[lat,lon], popup=label ,icon=folium.Icon(color=color, icon='ok-sign'),).add_to(m)

#'''

m.save('admits_map_Spring2019.html')
#m.save('admits_map_Spring2018.html')
    
m